In [1]:
from dataloader import DataLoader
import pandas as pd
import numpy as np

In [2]:
loader = DataLoader()

In [3]:
loader.load_sells_data_from_file('sprzedaz.csv')
df_filtered = loader = loader.prepare_sells_data()



In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# 1. Kolumny miesięczne
miesiace_kolumny = [str(i) for i in range(96, 0, -1)]

# 2. Agregacja do poziomu produktu
df_agg = df_filtered.groupby("KodProduktu")[miesiace_kolumny].sum().reset_index()

# 3. Przekształcenie na długi format
df_long = df_agg.melt(id_vars="KodProduktu", var_name="miesiac_ago", value_name="sprzedaz")
df_long["miesiac_ago"] = df_long["miesiac_ago"].astype(int)
df_long["czas"] = 97 - df_long["miesiac_ago"]

# 4. Tworzenie lagów (1-36)
lags = list(range(1, 37))
for lag in lags:
    df_long[f"lag_{lag}"] = df_long.groupby("KodProduktu")["sprzedaz"].shift(lag)

# 5. Cechy dodatkowe
# rolling, trend, std
df_long["rolling_mean_3"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(window=3).mean())
df_long["rolling_mean_12"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(window=12).mean())

# Trendy
df_long["trend_3"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.diff(3))
df_long["trend_12"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.diff(12))

# Odchylenia
df_long["std_3"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(3).std())
df_long["std_12"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(12).std())

# Sumy roczne
for i in range(1, 4):
    df_long[f"suma_rok_{i}"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(
        lambda x: x.shift(i * 12).rolling(12).sum()
    )

# 6. Usunięcie braków
df_lags = df_long.dropna().copy()

# 7. Kodowanie produktu
le = LabelEncoder()
df_lags["KodProduktu_enc"] = le.fit_transform(df_lags["KodProduktu"])

# 8. Dane treningowe
cechy_dodatkowe = [
    "rolling_mean_3", "rolling_mean_12", 
    "trend_3", "trend_12", 
    "std_3", "std_12",
    "suma_rok_1", "suma_rok_2", "suma_rok_3"
]
X = df_lags[[f"lag_{l}" for l in lags] + cechy_dodatkowe + ["KodProduktu_enc"]]
y = df_lags["sprzedaz"]

# 9. Trening modelu
model = xgb.XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1, random_state=42)
model.fit(X, y)

# 10. Prognozowanie dla jednego produktu
kod = "3224.011"
kod_enc = le.transform([kod])[0]

df_kod = df_lags[df_lags["KodProduktu"] == kod].sort_values("czas", ascending=False)
last_values = df_kod.head(36)["sprzedaz"].tolist()[::-1]  # najnowsze jako ostatnie

preds = []
for i in range(3):
    lag_features = last_values[-36:]
    rolling_mean_3 = pd.Series(last_values[-3:]).mean()
    rolling_mean_12 = pd.Series(last_values[-12:]).mean()
    trend_3 = last_values[-1] - last_values[-4]
    trend_12 = last_values[-1] - last_values[-13] if len(last_values) >= 13 else 0
    std_3 = pd.Series(last_values[-3:]).std()
    std_12 = pd.Series(last_values[-12:]).std()

    suma_rok_1 = sum(last_values[-12:])
    suma_rok_2 = sum(last_values[-24:-12])
    suma_rok_3 = sum(last_values[-36:-24])

    x_input = lag_features + [
        rolling_mean_3, rolling_mean_12,
        trend_3, trend_12,
        std_3, std_12,
        suma_rok_1, suma_rok_2, suma_rok_3,
        kod_enc
    ]

    y_pred = model.predict([x_input])[0]
    preds.append(y_pred)
    last_values.append(y_pred)

# 11. Wynik
for i, p in enumerate(preds):
    print(f"Prognozowana sprzedaż produktu {kod} za miesiąc {i+1} od teraz: {p:.2f}")


Prognozowana sprzedaż produktu 3224.011 za miesiąc 1 od teraz: -11.10
Prognozowana sprzedaż produktu 3224.011 za miesiąc 2 od teraz: 33.99
Prognozowana sprzedaż produktu 3224.011 za miesiąc 3 od teraz: 71.41
